In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.mol_opt import MolOpt
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time
import torch.nn.functional as F

/home/octav/gitrepos/tum-thesis


In [6]:
model_type = "molemb"
sys.argv = ["", "-cuda", "-model_type", model_type, "-one_batch_train"]
args = get_args()
outdir_suffix = "dev3"
args.n_epochs = 10000 
args.init_model = "{}-base-longer-softmax5".format(model_type)
# args.init_model = "{}-test1".format(model_type)
args.init_decoder_model = "{}_decode".format(args.init_model)
args.output_dir = "mol_opt/output_{}/{}".format(outdir_suffix, args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}/{}".format(outdir_suffix, args.init_model)
args.batch_size = 50 

args.penalty_gumbel = False 

args.scale_lambdas = True
args.connectivity = True 
args.valency = True 
args.euler_characteristic_penalty = True
args.conn_lambda_start = 0.001
args.conn_lambda_end = 30
args.conn_lambda_epochs_start = 4000
args.conn_lambda_epochs_end = 14000
args.valency_lambda_start = 0.001
args.valency_lambda_end = 50
args.valency_lambda_epochs_start = 4000
args.valency_lambda_epochs_end = 14000
args.euler_lambda_start = 0.001 
args.euler_lambda_end = 20 
args.euler_lambda_epochs_start = 4000
args.euler_lambda_epochs_end = 14000
args.tau_start = 1
args.tau_end = 0.001
args.tau_epochs_start = 1
args.tau_epochs_end = 16000


args.conn_penalty_function = "capped_logdet" 

print(args, '\n')

Namespace(N_transformer=6, agg_func='sum', batch_norm=False, batch_size=50, conn_lambda_end=30, conn_lambda_epochs_end=14000, conn_lambda_epochs_start=4000, conn_lambda_start=0.001, conn_penalty_function='capped_logdet', connectivity=True, connectivity_hard=False, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=True, euler_lambda_end=20, euler_lambda_epochs_end=14000, euler_lambda_epochs_start=4000, euler_lambda_start=0.001, ffn_activation='LeakyReLU', init_decoder_model='molemb-base-longer-softmax5_decode', init_model='molemb-base-longer-softmax5', linear_out=False, max_num_atoms=70, model_type='molemb', n_epochs=10000, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=200, n_labels=1, n_layers=5, one_batch_train=True, ot_solver='emd', output_dir='mol_opt/output_dev3/molemb-base-longer-softmax5', pc_hidden=150, penalty_gumbel=False, pred_hidden=150, scale_lambdas=True, sin

In [7]:
# train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, same_number_atoms = True)
train_data_loader = get_loader("molgen/data/chembl50", "train", args.batch_size, same_number_atoms = True)
# val_data_loader = get_loader("iclr19-graph2graph/data/qed", "valid", 36, False)

In [ ]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = None)

Found previous model mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5438, epoch 5438. Overwriting args.
Epoch: 5439
FGW torch.Size([29508, 5]) 5.797675112262368e-05
Penalty params: tau=0.09553 conn_l=0.00440 val_l=0.00474 euler_l=0.00415 epoch=5439 mode=[1 1 1] conn=True euler=True val=True
Losses Epoch 5439, train
 fgw:0.1530275
 conn_penalty:0.0000000
 val_penalty:0.0000000
 euler_penalty:0.0000000
 total:0.1530275
Epoch duration: 1.0747871398925781
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5439
Epoch: 5440
FGW torch.Size([29508, 5]) 5.442609472083859e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.09422529488801956
bonds_logits grad torch.Size([29508, 5]) 0.03705644607543945
Penalty params: tau=0.09549 conn_l=0.00441 val_l=0.00474 euler_l=0.00416 epoch=5440 mode=[1 1 1] conn=True euler=True val=True
Losses Epoch 5440, train
 fgw:0.1487816
 conn_penalty:1.0625919
 v

Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5452
Epoch: 5453
FGW torch.Size([29508, 5]) 5.7767421822063625e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.04366273432970047
bonds_logits grad torch.Size([29508, 5]) 0.01302746869623661
Penalty params: tau=0.09495 conn_l=0.00447 val_l=0.00481 euler_l=0.00421 epoch=5453 mode=[1 1 1] conn=True euler=True val=True
Losses Epoch 5453, train
 fgw:0.1599502
 conn_penalty:0.2486485
 val_penalty:0.7649065
 euler_penalty:0.5381217
 total:0.1670008
Epoch duration: 2.196026086807251
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5453
Epoch: 5454
FGW torch.Size([29508, 5]) 5.587680789176375e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.06360986828804016
bonds_logits grad torch.Size([29508, 5]) 0.01865849643945694
Penalty params: tau=0.09491 conn_l=0.00447 val_l=0.00482 euler_l=0.00422 epoch=5454 mode=[1 1 1] conn=True eule

Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5466
Epoch: 5467
FGW torch.Size([29508, 5]) 5.92558499192819e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.04460485279560089
bonds_logits grad torch.Size([29508, 5]) 0.010921712964773178
Penalty params: tau=0.09438 conn_l=0.00453 val_l=0.00489 euler_l=0.00427 epoch=5467 mode=[1 1 1] conn=True euler=True val=True
Losses Epoch 5467, train
 fgw:0.1484570
 conn_penalty:0.1979175
 val_penalty:0.3745924
 euler_penalty:0.4772091
 total:0.1532172
Epoch duration: 2.0818800926208496
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5467
Epoch: 5468
FGW torch.Size([29508, 5]) 6.088666123105213e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.03270299360156059
bonds_logits grad torch.Size([29508, 5]) 0.0067437742836773396
Penalty params: tau=0.09434 conn_l=0.00454 val_l=0.00489 euler_l=0.00428 epoch=5468 mode=[1 1 1] conn=True eu

Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5480
Epoch: 5481
FGW torch.Size([29508, 5]) 5.725583469029516e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.039297815412282944
bonds_logits grad torch.Size([29508, 5]) 0.00872714538127184
Penalty params: tau=0.09381 conn_l=0.00460 val_l=0.00496 euler_l=0.00433 epoch=5481 mode=[1 1 1] conn=True euler=True val=True
Losses Epoch 5481, train
 fgw:0.1485295
 conn_penalty:0.0184872
 val_penalty:0.2062632
 euler_penalty:0.4388417
 total:0.1515349
Epoch duration: 2.006822347640991
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5481
Epoch: 5482
FGW torch.Size([29508, 5]) 5.740796768805012e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.042022500187158585
bonds_logits grad torch.Size([29508, 5]) 0.010360687971115112
Penalty params: tau=0.09377 conn_l=0.00460 val_l=0.00496 euler_l=0.00433 epoch=5482 mode=[1 1 1] conn=True eu

Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5494
Epoch: 5495
FGW torch.Size([29508, 5]) 5.678645538864657e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.033868782222270966
bonds_logits grad torch.Size([29508, 5]) 0.008482103236019611
Penalty params: tau=0.09324 conn_l=0.00467 val_l=0.00504 euler_l=0.00439 epoch=5495 mode=[1 1 1] conn=True euler=True val=True
Losses Epoch 5495, train
 fgw:0.1415140
 conn_penalty:0.0882019
 val_penalty:0.2821844
 euler_penalty:0.4372014
 total:0.1452623
Epoch duration: 2.064791679382324
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5495
Epoch: 5496
FGW torch.Size([29508, 5]) 5.590678483713418e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.04443540796637535
bonds_logits grad torch.Size([29508, 5]) 0.012194957584142685
Penalty params: tau=0.09320 conn_l=0.00467 val_l=0.00504 euler_l=0.00440 epoch=5496 mode=[1 1 1] conn=True eu

...

Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5591
Epoch: 5592
FGW torch.Size([29508, 5]) 5.937967580393888e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.02899552509188652
bonds_logits grad torch.Size([29508, 5]) 0.00500837154686451
Penalty params: tau=0.08942 conn_l=0.00516 val_l=0.00559 euler_l=0.00483 epoch=5592 mode=[1 1 1] conn=True euler=True val=True
Losses Epoch 5592, train
 fgw:0.1509536
 conn_penalty:0.0000000
 val_penalty:0.3717949
 euler_penalty:0.2124301
 total:0.1540564
Epoch duration: 2.1180002689361572
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5592
Epoch: 5593
FGW torch.Size([29508, 5]) 5.845351188327186e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.02716551348567009
bonds_logits grad torch.Size([29508, 5]) 0.005257737822830677
Penalty params: tau=0.08938 conn_l=0.00516 val_l=0.00560 euler_l=0.00484 epoch=5593 mode=[1 1 1] conn=True eul

Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5605
Epoch: 5606
FGW torch.Size([29508, 5]) 5.6779987062327564e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.042964618653059006
bonds_logits grad torch.Size([29508, 5]) 0.007826549932360649
Penalty params: tau=0.08888 conn_l=0.00523 val_l=0.00568 euler_l=0.00490 epoch=5606 mode=[1 1 1] conn=True euler=True val=True
Losses Epoch 5606, train
 fgw:0.1482588
 conn_penalty:0.0313319
 val_penalty:0.1821251
 euler_penalty:0.3862260
 total:0.1513467
Epoch duration: 2.3201980590820312
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5606
Epoch: 5607
FGW torch.Size([29508, 5]) 5.424979462986812e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.08276399224996567
bonds_logits grad torch.Size([29508, 5]) 0.021658608689904213
Penalty params: tau=0.08884 conn_l=0.00524 val_l=0.00568 euler_l=0.00491 epoch=5607 mode=[1 1 1] conn=True 

Penalty params: tau=0.08838 conn_l=0.00530 val_l=0.00576 euler_l=0.00496 epoch=5619 mode=[1 1 1] conn=True euler=True val=True
Losses Epoch 5619, train
 fgw:0.1437312
 conn_penalty:0.0792796
 val_penalty:0.4431963
 euler_penalty:0.3883333
 total:0.1486265
Epoch duration: 2.322674512863159
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5619
Epoch: 5620
FGW torch.Size([29508, 5]) 5.62752247788012e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.03583434969186783
bonds_logits grad torch.Size([29508, 5]) 0.0073998006992042065
Penalty params: tau=0.08835 conn_l=0.00531 val_l=0.00576 euler_l=0.00497 epoch=5620 mode=[1 1 1] conn=True euler=True val=True
Losses Epoch 5620, train
 fgw:0.1524222
 conn_penalty:0.0368080
 val_penalty:0.4425761
 euler_penalty:0.2198048
 total:0.1562572
Epoch duration: 2.3158445358276367
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molem

Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5632
Epoch: 5633
FGW torch.Size([29508, 5]) 5.72454919165466e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.03221499174833298
bonds_logits grad torch.Size([29508, 5]) 0.006972859613597393
Penalty params: tau=0.08785 conn_l=0.00538 val_l=0.00585 euler_l=0.00503 epoch=5633 mode=[1 1 1] conn=True euler=True val=True
Losses Epoch 5633, train
 fgw:0.1464959
 conn_penalty:0.0000000
 val_penalty:0.3407780
 euler_penalty:0.2391607
 total:0.1496888
Epoch duration: 2.2094404697418213
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5633
Epoch: 5634
FGW torch.Size([29508, 5]) 5.6149379815906286e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.04183272272348404
bonds_logits grad torch.Size([29508, 5]) 0.010744872502982616
Penalty params: tau=0.08781 conn_l=0.00538 val_l=0.00585 euler_l=0.00504 epoch=5634 mode=[1 1 1] conn=True eu

Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5646
Epoch: 5647
FGW torch.Size([29508, 5]) 5.628274448099546e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.030757760629057884
bonds_logits grad torch.Size([29508, 5]) 0.006316610611975193
Penalty params: tau=0.08732 conn_l=0.00546 val_l=0.00594 euler_l=0.00510 epoch=5647 mode=[1 1 1] conn=True euler=True val=True
Losses Epoch 5647, train
 fgw:0.1485809
 conn_penalty:0.0000000
 val_penalty:0.1771322
 euler_penalty:0.3539459
 total:0.1514361
Epoch duration: 2.1863934993743896
Saving model, do not interrupt...
Saved at mol_opt/output_dev3/molemb-base-longer-softmax5/model_molemb-base-longer-softmax5_5647
Epoch: 5648
FGW torch.Size([29508, 5]) 5.60730550205335e-05
bonds_prob_conn grad torch.Size([29508, 5]) 0.04839365556836128
bonds_logits grad torch.Size([29508, 5]) 0.013214572332799435
Penalty params: tau=0.08728 conn_l=0.00546 val_l=0.00594 euler_l=0.00511 epoch=5648 mode=[1 1 1] conn=True eu

In [ ]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = None)

In [6]:
for i in train_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break
    
x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
# yhat_labels = molopt_decoder.discretize(*yhat_logits)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

from mol_opt.ot_utils import FGW 
fgw_loss = FGW(alpha = 0.5)
fgw_loss(*pred_pack, tau = 1)

ArgumentError: Python argument types in
    rdkit.Chem.rdmolops.Kekulize(NoneType)
did not match C++ signature:
    Kekulize(RDKit::ROMol {lvalue} mol, bool clearAromaticFlags=False)

In [30]:
torch.optim.Adam([torch.autograd.Variable(torch.Tensor([0.]))]).param_groups[0]['lr']

0.001

In [14]:
0.9 ** (1/5)

0.9791483623609768

In [28]:
vars(args)

{'cuda': True,
 'output_dir': 'mol_opt/output_dev1/pointwise-test5',
 'tb_logs_dir': 'mol_opt/logs_dev1/pointwise-test5',
 'init_model': 'pointwise-test5',
 'init_decoder_model': 'pointwise-test5_decode',
 'task': 'qed',
 'model_type': 'pointwise',
 'one_batch_train': True,
 'batch_size': 50,
 'pred_hidden': 150,
 'pc_hidden': 100,
 'ffn_activation': 'LeakyReLU',
 'n_epochs': 1000,
 'dim_tangent_space': 40,
 'n_layers': 5,
 'n_hidden': 250,
 'n_ffn_hidden': 100,
 'linear_out': False,
 'dropout_gcn': 0.0,
 'dropout_ffn': 0.0,
 'agg_func': 'sum',
 'batch_norm': False,
 'N_transformer': 6,
 'n_ffn_transformer': 100,
 'n_heads_transformer': 10,
 'dropout_transformer': 0.1,
 'ot_solver': 'emd',
 'sinkhorn_entropy': 0.1,
 'sinkhorn_max_it': 10000,
 'connectivity': False,
 'valency': True,
 'euler_characteristic_penalty': True,
 'annealing_rate': 0.002,
 'connectivity_lambda': 0,
 'valency_lambda': 0,
 'euler_lambda': 1,
 'connectivity_hard': False,
 'valency_hard': False,
 'scale_lambdas': F

In [32]:
from tensorboardX import SummaryWriter
writer = SummaryWriter("/home/octav/gitrepos/tum-thesis/mol_opt/dev7")

In [31]:
tags = [
    "pointwise10-dev8/train_avg_euler_error/mean",
    "pointwise10-dev8/train_avg_euler_error/bot_band",
    "pointwise10-dev8/train_avg_euler_error/top_band"
]
# writer.add_custom_scalars_marginchart(tags)


In [2]:
# writer.add_scalar("plm2/plm2", 0, 1)
# writer.add_scalar("plm2/plm2", 1, 2)
# writer.add_scalar("plm2/plm2", 0, 3)
# writer.add_scalar("plm2/plm2", 1, 4)
tags = ["plm", "plm", "plm"]
# layout = {"plm" : {"plm" : ["Margin", tags]}}
layout = {"plm1" : {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]},
          "plm2": {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]}}
writer.add_custom_scalars(layout)

In [33]:
writer._get_file_writer().add_summary()

TypeError: add_summary() missing 1 required positional argument: 'summary'

In [34]:
writer.logdir

'/home/octav/gitrepos/tum-thesis/mol_opt/dev6'

In [11]:
writer.close()

In [4]:
import tensorboardX
tensorboardX.__version__

'2.1'

In [6]:
from tensorboardX.summary import custom_scalars

In [16]:
writer._get_file_writer().add_summary(custom_scalars(layout), global_step = 3)
writer.flush()

In [34]:
writer.add_scalars('plm', {'plm0' : 0.0, 'plmt' : 0.1, 'plmb' : -0.1}, 3)
writer.flush()

In [ ]:
tau = 1
symbols_nll, charges_nll, bonds_nll = F.gumbel_softmax(tau = tau, dim=1, logits = symbols_logits), F.gumbel_softmax(tau=tau,dim=1,logits=charges_logits), F.gumbel_softmax(tau=tau,dim=1, logits = bonds_logits)

In [ ]:
np.sort(symbols_nll.mean(axis = 0).detach().numpy())

In [ ]:
list(molopt.opt0.parameters())[0]

In [ ]:
molopt.state_dict()

In [ ]:
model_dict = torch.load('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8')

In [ ]:
model_dict["model"]['opt0.weight']

In [ ]:
molopt2 = MolOpt(args)

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt2.load_state_dict(model_dict["model"])

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt3,args3 = load_model('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8', MolOpt, args.device)

In [ ]:
list(molopt3.opt0.parameters())[0]

In [ ]:
fgw_loss(*pred_pack, tau = 1)